In [2]:
import pandas as pd

In [3]:
df = pd.read_csv('files/all_claims_files.csv') # read consolidated claims file

In [4]:
# Add column for the file year since each row is a unique patient/year combination
df['year'] = df.file_name.map(lambda x: int(x[6:10]))

In [5]:
all_cols = df.columns # all columns of the Data Frame

In [7]:
payment_cols = ['MEDREIMB_IP', 'BENRES_IP', 'PPPYMT_IP', 'MEDREIMB_OP', 'BENRES_OP',
       'PPPYMT_OP', 'MEDREIMB_CAR', 'BENRES_CAR', 'PPPYMT_CAR'] # Only select the payment-related columns

In [9]:
id_vars = ['DESYNPUF_ID', 'year'] # Use the DESYNPUF_ID/year as a unique key for the row
# Alternatively, just use the DESYNPUF_ID as a unique key for the patient
# Use list comprehension to select all non-payment related columns
# Alternative: id_vars = [x for x in all_cols if x not in payment_cols]

In [10]:
df[id_vars].drop_duplicates().shape # Verify uniqueness

(6000, 2)

In [11]:
df_mlt = pd.melt(df, id_vars=id_vars, value_vars=payment_cols) # Melt the data!

In [14]:
# Split variable into two parts
df_mlt['payer'] = df_mlt['variable'].map(lambda x: x.split('_')[0])
df_mlt['service'] = df_mlt['variable'].map(lambda x: x.split('_')[1])

In [15]:
# Now can easily pivot in different ways

In [16]:
pd.crosstab(df_mlt.year, df_mlt.service, values=df_mlt.value, aggfunc='sum', normalize=True)

service,CAR,IP,OP
year,,,
2008,0.112737,0.202367,0.068266
2009,0.126575,0.196574,0.080897
2010,0.078089,0.091813,0.042680


In [17]:
df_mlt.pivot_table(index='year', columns='service', values='value', aggfunc='sum', margins=True)

service,CAR,IP,OP,All
year,,,,
2008,3231130,5799978,1956540,10987648
2009,3627740,5633954,2318570,11580264
2010,2238090,2631430,1223250,6092770
All,9096960,14065362,5498360,28660682
